In [7]:
import json
import requests

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool

from langchain.agents import create_agent
#from dotenv import load_dotenv
#load_dotenv()


In [8]:
from langchain_openai import ChatOpenAI

gpt4o_llm = ChatOpenAI(model="gpt-4o", temperature=0)
response = gpt4o_llm.invoke("Hola como estas?")
response.pretty_print()

================================== Ai Message ==================================

¡Hola! Estoy bien, gracias. ¿Y tú? ¿En qué puedo ayudarte hoy?


In [9]:
# -------------------------
# Tool real (Open-Meteo)
# -------------------------
@tool
def get_weather(city: str) -> str:
    """Obtiene la temperatura actual (°C) de una ciudad usando Open-Meteo."""
    geo = requests.get(
        "https://geocoding-api.open-meteo.com/v1/search",
        params={"name": city, "count": 1, "language": "es", "format": "json"},
        timeout=15,
    ).json()

    if not geo.get("results"):
        return json.dumps({"error": f"No pude geocodificar '{city}'."}, ensure_ascii=False)

    r0 = geo["results"][0]
    lat, lon = r0["latitude"], r0["longitude"]
    resolved = f'{r0.get("name", city)}, {r0.get("country", "")}'.strip(", ")

    forecast = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={"latitude": lat, "longitude": lon, "current_weather": True, "temperature_unit": "celsius"},
        timeout=15,
    ).json()

    cw = forecast.get("current_weather")
    if not cw:
        return json.dumps({"error": "No pude obtener current_weather."}, ensure_ascii=False)

    return json.dumps(
        {
            "city_input": city,
            "city_resolved": resolved,
            "temperature_c": cw.get("temperature"),
            "time": cw.get("time"),
            "source": "open-meteo",
        },
        ensure_ascii=False,
    )


In [10]:
# -------------------------
# CASO A: LLM "solo"
# -------------------------
def llm_solo():
    model = ChatOpenAI(model="gpt-4o", temperature=0)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system",
             "Eres un asistente. Responde en español.\n"
             "Tarea: decir la temperatura actual de Medellín (°C).\n"
             "Regla: NO tienes datos en tiempo real. Debes decirlo explícitamente y dar una estimación razonable."),
            ("human", "{question}"),
        ]
    )

    chain = prompt | model
    out = chain.invoke({"question": "¿Cuál es la temperatura actual en Medellín (°C)?"})

    print("\n=== CASO A: LLM sin tools (solo prompt) ===")
    print(out.content)

llm_solo()


=== CASO A: LLM sin tools (solo prompt) ===
Lo siento, no tengo acceso a datos en tiempo real, por lo que no puedo proporcionar la temperatura actual en Medellín. Sin embargo, puedo ofrecerte una estimación basada en el clima típico de la ciudad. Medellín, conocida como la "Ciudad de la Eterna Primavera", suele tener temperaturas agradables durante todo el año, generalmente entre 18°C y 28°C. En un día típico, la temperatura podría estar alrededor de 22°C a 24°C. Te recomendaría consultar una fuente de información meteorológica actualizada para obtener la temperatura exacta.


In [11]:
# -------------------------
# CASO B: Agente (prompt + tools) SIN loop explícito
# - create_agent maneja internamente el ciclo tool-calling/stop-condition
# -------------------------
def agente_con_tool():
    model = ChatOpenAI(model="gpt-4o", temperature=0)
    tools = [get_weather]

    # ✅ “Toda la instrucción en un prompt” (system_prompt)
    system_prompt = (
        "Eres un agente. Responde en español.\n"
        "Objetivo: dar la temperatura ACTUAL de Medellín (°C).\n"
        "Regla: para datos en tiempo real DEBES usar la herramienta get_weather.\n"
        "Si la herramienta falla, dilo y da una aproximación."
    )

    graph = create_agent(
        model=model,
        tools=tools,
        system_prompt=system_prompt,  # ✅ este es el parámetro correcto en 1.x
        debug=True,                   # opcional: logs del runtime (útil en clase)
    )

    # ✅ invocación “messages-based”
    result = graph.invoke(
        {"messages": [HumanMessage(content="¿Cuál es la temperatura actual en Medellín (°C)?")]}
    )

    # result["messages"] trae todo el historial; el último suele ser la respuesta final
    last = result["messages"][-1]
    print(getattr(last, "content", last))

agente_con_tool()

[values] {'messages': [HumanMessage(content='¿Cuál es la temperatura actual en Medellín (°C)?', additional_kwargs={}, response_metadata={}, id='51147eef-dcb7-4f58-8aa2-a5481c26f03b')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 122, 'total_tokens': 138, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ad98c18a04', 'id': 'chatcmpl-D8DEJmONysaRr15ocnPXFM8gNM5LV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c4ee0-5818-7083-be3f-7b602f9fc6d8-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Medellín'}, 'id': 'call_WRy8iZyw3uywQCBDGo1y0PH3', 'type': 'tool_call'}], inval